In [28]:
import nest_asyncio
import aiohttp
import asyncio
import nest_asyncio
import re
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from os import getenv
from openai import OpenAI
import pathlib
import textwrap
# import google.generativeai as genai
import time
import requests
import ast
import json
from sklearn.metrics.pairwise import cosine_similarity
import re
from llms import chatGPT
from llms import gemini


x_chat = chatGPT()
x_gemini = gemini()

In [229]:
df = pd.read_csv("../output/data/data_post_chat_gpt.csv")
df = df[df["headquarters location"].str.contains("United States")]
df = df[pd.to_datetime(df['founded date'])>='11/30/2022']

In [230]:
df = df[df["organization name"].isin(["AI Mavericks","AI Teaches Me from BAM Software Development","AI Virtual Staging"])]


In [231]:
generated_description_prompt = """
Tell me about ${company}'s product.

Next, rate your confidence in the description on a scale from 1 (not confident) to 10 (highly confident). Remember to follow the example format!

**EXAMPLES BELOW**

Company: Genmo
Website: https://www.genmo.ai/
Description: Genmo is a free tool that creates videos and images using artificial intelligence. Genmo can also generate creative text formats of text content, like poems, code, scripts, musical pieces, email, letters, etc.

Company: OnePane
Website: https://www.onepane.ai/
Description: Onepane is a company that offers an AI companion for enhanced DevOps & SRE efficiency. Onepane offers a GenAI solution providing unparalleled unified insights and control over your Cloud resources. Onepane helps with root cause analysis, cloud governance, and optimization strategies.

**YOUR TURN**

Company: $company
Website: $website
Current Product Description: $description
________________________
Description (in two sentences):
Confidence Interval:
Reasoning:
________________________
"""



In [232]:
filter_prompt = """
On a scale of 1-10, Is the goal of the product of $company ($website) to be used by a task & occupation in the US workforce?
Description of $company: $description
Please give a specific number and your reasoning why.
Automation Rating: --
Reasoning: --
"""

In [233]:
parsed_description_prompt = """
Your role is to describe the jobs/tasks, industries, and customers that $company is targeting. Then, provide a confidence interval (1-10) on how sure you are about the response. 

A response should include:
- tasks/jobs being automated
- the industry that the startup applies to
- specific customers using the tool. DO NOT INCLUDE AN EXPLANATION

Remember to follow the example format!

**EXAMPLES BELOW**

Company: Petville
Website: petville.co/pricing/biz
Description: Petville Global is a B2B CRM SaaS platform that utilizes advanced technologies like AI/ML and neural net to streamline and expand operations for pet businesses and veterinary clinics both locally and globally. The platform offers deep data analytics and marketing tools, helping businesses save an average of 22% on CRM and vet tech costs.
Tasks/Jobs: Data analysis, Marketing automation, Appointment scheduling, Inventory management
Industry: Customer management
Customers: Veterinarians, Pet Businesses
Confidence Interval: 8
Reasoning: The description aligns well with the company's focus on CRM for pet-related businesses and includes specific tasks and industries that match the provided information.

Company: Thunder
Website: thundercompute.com
Description: Thunder is a decentralized, peer-to-peer cloud computing platform designed to democratize access to GPUs and address the persistent GPU shortage. It provides a solution for individuals and businesses seeking high-performance computing power, enabling them to leverage unused GPU resources from around the world.
Tasks/Jobs: GPU resource allocation, Access to unused GPUs, Distributed computing tasks
Industry: Cloud Computing
Customers: Developers, GPU Owners
Confidence Interval: 9
Reasoning: The platform's focus on GPU allocation and decentralized computing is clear, and the target customers and industry are well-defined and specific.

Company: NonprofitsHQ
Website: www.nonprofitshq.com
Description: NonprofitsHQ is a software suite designed for nonprofits that utilizes AI to automate tasks, manage operations, and improve efficiency, ultimately saving organizations time and resources.
Tasks/Jobs: Fundraising management, Donor relationship management, Grant writing, Event planning
Industry: Non-profit management
Customers: Non-profit organizations
Confidence Interval: 7
Reasoning: The description covers essential aspects of nonprofit management and identifies specific tasks and customers, but there could be more detail on the AI's role in these tasks.

**YOUR TURN**

Company: $company
Website: $website
Current Product Description: $description
________________________
Tasks/Jobs: (comma separated list of 4, short)
Industry: 
Customers: (comma separated list)
Confidence Interval: 
Reasoning: 
________________________
"""


In [280]:
examples_prompt = """
Imagine you're explaining the product from $company to someone unfamiliar with it. Provide 3 concise examples of how this product could be used in a real-world scenario. Keep the descriptions general and avoid mentioning the company name.

For each example, describe:
- Example of when someone would use the product
- Job being automated and the task that the tool replaces (follow ONET task statements for example on how to format a task: https://www.onetcenter.org/dictionary/20.1/excel/task_statements.html)
- Next, rate your confidence in example task on a scale from 1 (not confident) to 10 (highly confident). 


Remember to follow the example format!!!!

**EXAMPLES BELOW**

Company: Blanc
Website: tryblanc.ai
Blanc is a compliance automation platform designed for fintech companies. It helps streamline regulatory compliance processes by providing a centralized hub for managing policies, monitoring activities, and generating reports.
________________
Example 1: A compliance officer at a fintech company uses Blanc to automate the process of creating and updating compliance policies, ensuring all documents are current and accessible to relevant team members.
JOB automated 1: Compliance Officers that verify that all firm and regulatory policies and procedures have been documented, implemented, and communicated.
JOB 1: Compliance Officers

Example 2: A fintech company uses Blanc to generate automated compliance reports for regulatory audits, ensuring all necessary documentation is readily available and organized.
JOB automated 2: Compliance Officers Prepare reports of activities, evaluations, recommendations, or decisions.
JOB 2: Compliance Officers

Example 3: A fintech company uses Blanc to monitor real-time transactions for potential compliance violations, triggering alerts and generating reports for further investigation.
JOB automated 3: Compliance Officers that identify compliance issues that require follow-up or investigation.
JOB 3: Compliance Officers
________________

Company: Aether
Website: aetherenergie.com/
Aether Energy is an AI-driven platform designed to simplify the process of rooftop solar installation for businesses, providing comprehensive support from project planning and financing to installation and ongoing maintenance. This platform aims to streamline and optimize the entire solar energy journey for installers. 
________________
Example 1: A solar installer uses Aether to quickly create detailed project plans for rooftop solar installations, including system size, panel placement, and wiring diagrams. 
automated 1: Solar Photovoltaic Installers that diagram layouts and locations for photovoltaic (PV) arrays and equipment, including existing building or site features.
JOB 1: Solar Photovoltaic Installers

Example 2: A business owner leverages Aether to secure financing for their rooftop solar project, providing them with customized loan options and streamlined application processes. 
JOB automated 2: Solar Photovoltaic Installers that prepare solar installation project proposals, quotes, budgets, or schedules.
JOB 2: Solar Photovoltaic Installers

Example 3: A solar installer uses Aether to manage the installation process, tracking materials, scheduling technicians, and coordinating with subcontractors, ensuring smooth project execution. 
JOB automated 3:  Solar Engery Installation Managers that monitor work of contractors and subcontractors to ensure projects conform to plans, specifications, schedules, or budgets.
JOB 3: Solar Energy Installation Managers
________________


**YOUR TURN**
Company:$company
Website: $website
Current Description: $description
$parsed_description
________________
Example 1: 
JOB automated 1:
JOB 1: 
Confidence Interval 1:
Reasoning 1:

Example 2: 
JOB automated 2:
JOB 2: 
Confidence Interval 2:
Reasoning 2:

Example 3:
JOB automated 3:
JOB 3: 
Confidence Interval 3:
Reasoning 3:
________________
"""

In [235]:
# Apply nest_asyncio
nest_asyncio.apply()


class prompting():
    def __init__(self):
        self.results_df = pd.DataFrame(columns=["organization name", "value"])

    def set_current_results_df(self, results_df):
        self.results_df = results_df

    async def iterate(self, df, prompt_template, args, value, batch_size=10, start=0, end=False):
        if end == False:
            end = len(df)
        self.results_df = pd.DataFrame(columns=["organization name", value])
        if value in list(df.columns):
            if start != 0:
                self.results_df = pd.concat([df[["organization name", value]].iloc[:start], self.results_df], axis=0)
            df = df.drop(columns=[value])

        batch_prompts = []
        batch_indices = []

        for i, row in list(df.iterrows())[start:end]:
            name = row['organization name']
            website = row['website']
            prompt = prompt_template

            for arg in args:
                prompt = prompt.replace(f"${arg[0]}", row[arg[1]])

            batch_prompts.append((i, name, prompt))
            batch_indices.append(i)

            if len(batch_prompts) >= batch_size:
                await self.process_batch(batch_prompts)
                batch_prompts = []

        if batch_prompts:
            await self.process_batch(batch_prompts)
            self.results_df.to_csv("../output/current_results_df_prompting.csv")

        df = df.merge(self.results_df, on='organization name', how='left')
        return df

    async def process_batch(self, batch_prompts):
        async with aiohttp.ClientSession() as session:
            tasks = []
            for i, name, prompt in batch_prompts:
                tasks.append(self.fetch_result(session, i, name, prompt))
            await asyncio.gather(*tasks)

    async def fetch_result(self, session, i, name, prompt):
        failure_count = 0
        while True:
            try:
                print(f"******************************\nProcessing {i}: {name}")
                result = await x_gemini.ask(session, prompt)
                if result == "N/A": break  # explicit material

                text = re.sub(r"#|#\s+|_|\*", "", result).strip()

                self.results_df.loc[i] = [name, text]

                    
                break
            except Exception as e:
                print(failure_count)
                failure_count += 1
                if failure_count > 10:
                    break
                print(f"Error processing {i}, {name}: {e}")
                await asyncio.sleep(20)

prompting_class = prompting()


## Generated Description

In [236]:
cols = ["generated_description", "generated_description_conf_interval", "generated_description_conf_interval_reasoning"]
args = [["company","organization name"], ["website","website"], ["description","description_all"]]

async def main(df):
    df = await prompting_class.iterate(df, generated_description_prompt, args, "generated_description_llm")
    df.to_csv('../output/df_with_generated_description.csv', index=False)
    return df

df = asyncio.run(main(df))

******************************
Processing 1073: AI Virtual Staging
******************************
Processing 1776: AI Mavericks
******************************
Processing 2102: AI Teaches Me from BAM Software Development


In [237]:
def extract_data(text):
    if pd.isnull(text): return {}
    text = text.replace("\n"," ").replace("  "," ").replace("*","").replace(" (two sentences)","").replace("/10","").replace("_","").replace("#","")
    pattern = r"^.*?\s*Description:?\s*(.*)Confidence Interval:\s*(\d+)\s*Reasoning:\s*(.*)$"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        description = match.group(1).strip()
        confidence_interval = match.group(2).strip()
        reasoning = match.group(3).strip()
        
        result = {
            "generated_description": description,
            "generated_description_conf_interval": confidence_interval,
            "generated_description_conf_interval_reasoning": reasoning
        }
        return result
        
    else:
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["generated_description_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)
len(df)

3

In [238]:
# df = pd.read_csv("../output/df_with_generated_description.csv", index_col=0).reset_index(drop=True)
# len(df)

In [239]:
df = df.dropna(subset=["generated_description"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

3

## Filter Prompt

In [240]:
cols = ["automation_rating"]
args = [["company","organization name"], ["website","website"], ["description","description_all"]]

async def main(df):
    df = await prompting_class.iterate(df, filter_prompt, args, "automation_rating_llm")
    df.to_csv('../output/df_with_filter.csv', index=False)
    return df

df = asyncio.run(main(df))

******************************
Processing 0: AI Virtual Staging
******************************
Processing 1: AI Mavericks
******************************
Processing 2: AI Teaches Me from BAM Software Development


In [241]:
for x in range(1,2):
    print(df.automation_rating_llm[x])

Automation Rating: 3

Reasoning: While AI Mavericks focuses on AI and business growth, the goal isn't directly to be used by a specific task or occupation in the US workforce. Their events provide information and education on AI implementation, which can indirectly impact various roles across different industries. However, their product is not an AI tool itself but a service that helps businesses understand and adopt AI. 

Here's why it's not a higher rating:

 Not a Tool: AI Mavericks doesn't produce AI software or tools for specific tasks. They facilitate knowledge sharing and networking around AI.
 Indirect Impact: While their events can influence how companies use AI, the impact on individual tasks or occupations is indirect and potentially broad. 
 Focus on Business Leaders: The target audience for AI Mavericks is leadership teams, not individual workers with specific tasks. 

Therefore, while AI Mavericks indirectly contributes to the use of AI in the workforce, their product isn

In [242]:
def extract_data(text):
    pattern = r"Automation Rating: (\d+)\s+Reasoning:\s+(.*)"

    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)

    if match:
        automation_rating = match.group(1).strip()
        automation_rating_reasoning = match.group(2).strip()

        result = {
            "automation_rating": automation_rating,
            "automation_rating_reasoning": automation_rating_reasoning
        }
        return result

    else:
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["automation_rating_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


,Unnamed: 0,organization name,full_description,industries,headquarters location,founded date,description,CB rank,headquarters region,postal code,...,last funding type,description_all,industries_parsed,generated_description_llm,generated_description,generated_description_conf_interval,generated_description_conf_interval_reasoning,automation_rating_llm,automation_rating,automation_rating_reasoning
0,1077,AI Virtual Staging,—,"Artificial Intelligence (AI), Property Managem...","San Francisco, California, United States",2023-02-01,Real estate virtual staging using AI,—,"San Francisco Bay Area, West Coast, Western US",—,...,—,Real estate virtual staging using AI —,"['Property Management', 'Real Estate', 'Real E...",AI Virtual Staging\n\nDescription (in two sent...,(in two sentences): AI Virtual Staging uses ar...,8,The website provides a clear visual demonstrat...,Automation Rating: 9\n\nReasoning:\n\nAI Virtu...,9,AI Virtual Staging directly targets a specific...
1,1783,AI Mavericks,AI Mavericks is an event management company th...,"Artificial Intelligence (AI), Consulting, Even...","Lehi, Utah, United States",2023-10-24,Event Management Company,—,Western US,—,...,—,Event Management Company AI Mavericks is an ev...,"['Consulting', 'Event Management']",Description (in two sentences): AI Mavericks i...,(in two sentences): AI Mavericks is an event m...,7,The provided description focuses on the compan...,Automation Rating: 3\n\nReasoning: While AI Ma...,3,While AI Mavericks focuses on AI and business ...
2,2113,AI Teaches Me from BAM Software Development,—,"Artificial Intelligence (AI), E-Learning, EdTech","Lowell, Arkansas, United States",2024-03-01,A revolution in personalized learning. Our cus...,—,Southern US,72745,...,—,A revolution in personalized learning. Our cus...,"['E-Learning', 'EdTech']",Description: AI Teaches Me by BAM Software Dev...,AI Teaches Me by BAM Software Development is a...,6,"While the provided description is vague, the w...",Automation Rating: 7\n\nReasoning:\n\nWhile th...,7,"While the product ""AI Teaches Me"" sounds promi..."


## Parsed Description

In [244]:
cols = ["parsed_description", "parsed_description_conf_interval", "parsed_description_conf_interval_reasoning", "Tasks/Jobs","Industry","Customers"]  
args = [["company","organization name"],["website","website"], ["description","generated_description"]]

async def main(df):
    df = await prompting_class.iterate(df, parsed_description_prompt, args, "parsed_description_llm")
    df.to_csv('../output/df_with_parsed_description.csv', index=False)
    return df

df = asyncio.run(main(df))




******************************
Processing 0: AI Virtual Staging
******************************
Processing 1: AI Mavericks
******************************
Processing 2: AI Teaches Me from BAM Software Development


In [245]:
def extract_data(text):
    if pd.isnull(text): return {}
    text = re.sub(r'\s+', ' ', text)  
    text = text.replace("_", "").replace("*", "").replace("#", "")
    text = re.sub(r'\s?\([^)]*\)', '', text)
    pattern = r".*?Tasks/Jobs:\s*(.*?)\s*Industry:\s*(.*?)\s*Customers:\s*(.*?)\s*Confidence Interval:\s*(.*?)\s*Reasoning:\s*(.*)"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        tasks_jobs = match.group(1).strip()
        industry = match.group(2).strip()
        customers = match.group(3).strip()
        confidence_interval = match.group(4).strip()
        reasoning = match.group(5).strip()
        
        result = {
            "parsed_description": 
            "Tasks/Jobs: " + tasks_jobs + "\n" + "Industry: " + industry + "\n" + "Customers: "+ customers,
            "Tasks/Jobs": tasks_jobs,
            "Industry": industry,
            "Customers": customers,
            "parsed_description_conf_interval": confidence_interval,
            "parsed_description_conf_interval_reasoning": reasoning
        }
        
        return result
    else:
        print(text)
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["parsed_description_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


In [246]:
len(df)

3

In [247]:
df = df.dropna(subset=["parsed_description"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

3

In [248]:
df.to_csv('../output/df_with_parsed_description.csv', index=False)

## Examples

In [281]:
cols = ["situation1", "situation1_conf_interval", "situation1_conf_interval_reasoning", "situation2", "situation2_conf_interval", "situation2_conf_interval_reasoning","situation3", "situation3_conf_interval", "situation3_conf_interval_reasoning",'Example1','Job1','Job1_title','Example2','Job2','Job2_title','Example3','Job3','Job3_title']
args = [["company","organization name"],["website","website"], ["description","generated_description"], ["parsed_description","parsed_description"]]


async def main(df):
    df = await prompting_class.iterate(df, examples_prompt, args, "examples_llm",1060)
    df.to_csv('../output/df_with_examples.csv', index=False)
    return df

df = asyncio.run(main(df))


******************************
Processing 0: AI Virtual Staging
******************************
Processing 1: AI Teaches Me from BAM Software Development


In [284]:

def extract_data(text):
    if pd.isnull(text): return {}
    # Normalize the text to ensure consistent whitespace and remove unwanted characters.
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace("*", "").replace("/10", "").replace("_", "").replace("#", "").replace("- ","")
    vals = {}
    i = 1
    while i <=3:
        # Regex pattern adjusted to handle fractions in confidence intervals like '9/10'
        pattern = rf"Example\s+{i}:\s+(.*?)\s+JOB\s+automated\s+{i}:\s+(.*?)\s+JOB\s+{i}:\s+(.*?)\s+Confidence\s+Interval\s+{i}:\s+(\d+(?:/\d+)?)\s+Reasoning\s+{i}:\s+(.*?)(?=\s*Example\s+{i + 1}:|$)"
        match = re.search(pattern, text, re.DOTALL)
        if not match:
            print(text+"\n\n")
            print(f"No matches found for Example {i}")  # Debug if no examples are found
            return {}
            break

        example_text, onet_job_automated, onet_job, confidence_interval, reasoning = match.groups()
        example_key = f"Example{i}"
        vals[example_key] = example_text.strip()
        vals[f"Job{i}"] = onet_job_automated.strip()
        vals[f"Job{i}_title"] = onet_job.strip()
        vals[f"situation{i}_conf_interval"] = confidence_interval.strip()
        vals[f"situation{i}_conf_interval_reasoning"] = reasoning.strip()

        i += 1  # Prepare to search for the next example
    return vals

for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])

results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["examples_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


In [285]:
df = df.dropna(subset=["Example1"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

2

In [286]:
df.to_csv('../output/df_with_examples.csv', index=False)

In [202]:
#RERUN STOPPED IN MIDDLE
# results_df = pd.read_csv("result.csv",index_col=0)
# df = pd.read_csv('../output/df_with_parsed_description.csv',index_col=0)
# prompting_class.set_current_results_df(results_df)
# df = df.merge(results_df, on='organization name', how='left')

In [25]:
# df = pd.read_csv('../output/results/df_with_examples.csv',index_col=0)


In [287]:
df.head()

,Unnamed: 0,organization name,full_description,industries,headquarters location,founded date,description,CB rank,headquarters region,postal code,...,Example2,Job2,Job2_title,situation2_conf_interval,situation2_conf_interval_reasoning,Example3,Job3,Job3_title,situation3_conf_interval,situation3_conf_interval_reasoning
0,1077,AI Virtual Staging,—,"Artificial Intelligence (AI), Property Managem...","San Francisco, California, United States",2023-02-01,Real estate virtual staging using AI,—,"San Francisco Bay Area, West Coast, Western US",—,...,Scenario: A homeowner is selling their home an...,Real Estate Agents that prepare visual present...,Real Estate Agents,8,AI virtual staging allows homeowners to experi...,Scenario: A real estate photographer is creati...,Photographers that photograph properties for s...,Photographers,7,AI virtual staging allows photographers to cre...
1,2113,AI Teaches Me from BAM Software Development,—,"Artificial Intelligence (AI), E-Learning, EdTech","Lowell, Arkansas, United States",2024-03-01,A revolution in personalized learning. Our cus...,—,Southern US,72745,...,Scenario: A professional seeking to upskill in...,"Trainers that plan, conduct, and evaluate trai...",Training and Development Specialists,8,While trainers still play a role in developing...,Scenario: An individual wanting to learn a new...,Language instructors that provide instruction ...,Language Instructors,7,The platform provides many of the functions of...


In [283]:
for i,x in df.iloc[:1].transpose().iterrows():
    print(i,x[0])

Unnamed: 0 1077
organization name AI Virtual Staging
full_description —
industries Artificial Intelligence (AI), Property Management, Real Estate, Real Estate Brokerage, Software
headquarters location San Francisco, California, United States
founded date 2023-02-01
description Real estate virtual staging using AI
CB rank —
headquarters region San Francisco Bay Area, West Coast, Western US
postal code —
estimated revenue —
operating status Active
exit date —
closed date —
company type For Profit
website www.virtualstaging.art
twitter —
facebook —
linkedin View on LinkedIn
email info@virtualstaging.art
phone number —
number of articles —
hub tags —
actively hiring —
num of sub orgs —
investor type —
investor stage —
number of founders —
founders —
number of employees —
total funding amount —
number of funding rounds —
last funding amount —
last funding type —
description_all Real estate virtual staging using AI —
industries_parsed ['Property Management', 'Real Estate', 'Real Estate Broke